In [6]:
import pickle
import sys
import numpy as np
from pathlib import Path

# 1. SETUP PATHS
# Use pathlib to safely find the file relative to the 'notebooks' folder
file_path = Path('..') / 'data' / 'raw' / 'crystal_graphs_dataset.pkl'
print(f"Attempting to load: {file_path.resolve()}")

# 2. DEFINE DUMMY CLASSES (The "Safety Net")
# If the pickle file looks for 'CrystalGraphDataset' or 'Graph' and can't find them,
# it will use this Placeholder class instead of crashing.
class Placeholder:
    def __init__(self, *args, **kwargs): pass
    def __setstate__(self, state): self.__dict__ = state

# Inject placeholders into the main namespace
if not hasattr(sys.modules['__main__'], 'CrystalGraphDataset'):
    setattr(sys.modules['__main__'], 'CrystalGraphDataset', Placeholder)
if not hasattr(sys.modules['__main__'], 'Graph'):
    setattr(sys.modules['__main__'], 'Graph', Placeholder)

# 3. LOAD THE DATA
try:
    with open(file_path, 'rb') as f:
        loaded_data = pickle.load(f)
    print("✅ Success: File loaded!")
except Exception as e:
    print(f"❌ Error loading file: {e}")
    raise e

# 4. INSPECT THE STRUCTURE
# We need to know exactly how nodes and edges are stored to write the next script.
if isinstance(loaded_data, dict):
    graphs = loaded_data.get('graphs', [])
    metadata = loaded_data.get('metadata', [])
    print(f"\nDataset contains {len(graphs)} graphs and {len(metadata)} metadata entries.")
    
    if len(graphs) > 0:
        g0 = graphs[0]
        print(f"\n--- GRAPH 0 STRUCTURE ---")
        print(f"Object Type: {type(g0)}")
        
        # Check Node Features (CRITICAL for Discretization)
        if hasattr(g0, 'nodes'):
            nodes = g0.nodes
            print(f"1. .nodes type: {type(nodes)}")
            
            # If list, print first element
            if isinstance(nodes, list) and len(nodes) > 0:
                feat = nodes[0]
                print(f"2. Node[0] Data: {feat}")
                print(f"3. Node[0] Type: {type(feat)}")
                if isinstance(feat, np.ndarray):
                    print(f"4. Feature shape: {feat.shape}")
            # If dict, print first key-value
            elif isinstance(nodes, dict) and len(nodes) > 0:
                first_key = list(nodes.keys())[0]
                print(f"2. Node[{first_key}] Data: {nodes[first_key]}")
        else:
            print("⚠️ Graph object has no '.nodes' attribute.")

        # Check Edges (CRITICAL for Subgraph Extraction)
        if hasattr(g0, 'edges'):
            edges = g0.edges
            print(f"5. .edges type: {type(edges)}")
            if hasattr(edges, '__iter__') and len(edges) > 0:
                print(f"6. Sample Edge: {list(edges)[0]}")
        elif hasattr(g0, 'adj'):
            print(f"5. Graph uses adjacency list (.adj) instead of .edges")
        else:
            print("⚠️ Graph object has no '.edges' attribute.")
            
        # Check Metadata
        if len(metadata) > 0:
            print(f"\n--- METADATA SAMPLE ---")
            print(metadata[0])
else:
    print(f"Unexpected data structure: {type(loaded_data)}")

Attempting to load: /home/npkamath/553Project/553ProjectGraphKernels/data/raw/crystal_graphs_dataset.pkl
✅ Success: File loaded!

Dataset contains 132 graphs and 132 metadata entries.

--- GRAPH 0 STRUCTURE ---
Object Type: <class 'networkx.classes.graph.Graph'>
1. .nodes type: <class 'networkx.classes.reportviews.NodeView'>
5. .edges type: <class 'networkx.classes.reportviews.EdgeView'>
6. Sample Edge: (0, 1)

--- METADATA SAMPLE ---
{'crystal_type': 'sc', 'noise_level': 0.0, 'sample_idx': 0, 'n_nodes': 3375, 'n_edges': 10125, 'system_size': 15, 'scale_factor': 1.0, 'ls': (4, 5, 6, 8, 10, 12), 'weight_threshold': 0.05, 'cutoff': 1.8}


In [32]:
import pickle
import random
import networkx as nx
import numpy as np
from pathlib import Path
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from grakel import Graph
from grakel.kernels import WeisfeilerLehmanOptimalAssignment

# --- CONFIGURATION ---
# Path logic: Go up one level from 'notebooks' to find 'data'
RAW_DATA_PATH = Path('..') / 'data' / 'raw' / 'crystal_graphs_dataset.pkl'

# Preprocessing Constants
NOISE_THRESHOLD = 0.31     # If noise > 0.3, label becomes 'Disordered'
NEIGHBOR_RADIUS = 1       # 1 = immediate neighbors only
N_BINS = 10               # Number of distinct "colors" for Minkowski values

# Sampling (CRITICAL for performance)
# We take 30 random nodes per graph. 
# 132 graphs * 30 nodes = ~4,000 training samples.
SAMPLES_PER_GRAPH = 5    
RANDOM_SEED = 42

# Model Constants
WL_ITERATIONS = 4         # Depth of the WL refinement (h parameter)
SVM_C = 1.0               # Regularization strength

print(f"Configuration loaded. Looking for data at: {RAW_DATA_PATH.resolve()}")

Configuration loaded. Looking for data at: /home/npkamath/553Project/553ProjectGraphKernels/data/raw/crystal_graphs_dataset.pkl


In [33]:
def get_node_feature_vector(node_attributes):
    """
    Manually extracts M4, M5, M6, M8, M10, M12 into a single numpy array.
    """
    # Define the specific keys we want to use as features
    # Based on your debug output
    feature_keys = ['M4', 'M5', 'M6', 'M8', 'M10', 'M12']
    
    features = []
    for key in feature_keys:
        if key in node_attributes:
            features.append(node_attributes[key])
        else:
            # Fallback if a key is missing (shouldn't happen in clean data)
            features.append(0.0)
            
    return np.array(features)

def process_graphs(graphs, metadata):
    random.seed(RANDOM_SEED)
    
    # --- 1. COLLECT ALL FEATURES ---
    print("Step 1: Collecting features from all graphs...")
    all_features = []

    for G in graphs:
        # Loop through every node and build the vector manually
        for _, data in G.nodes(data=True):
            vec = get_node_feature_vector(data)
            all_features.append(vec)
            
    all_features = np.array(all_features)
    print(f"   > Collected {len(all_features)} feature vectors.")
    
    # --- 2. TRAIN DISCRETIZER ---
    print(f"Step 2: Discretizing into {N_BINS} bins...")
    discretizer = KBinsDiscretizer(n_bins=N_BINS, encode='ordinal', strategy='kmeans')
    discretizer.fit(all_features)
    
    # --- 3. EXTRACT & LABEL SUBGRAPHS ---
    print("Step 3: Extracting subgraphs...")
    final_subgraphs = []
    final_labels = []
    
    for G, meta in zip(graphs, metadata):
        # A. Determine Class Label
        if meta['noise_level'] > NOISE_THRESHOLD:
            label = 'Disordered'
        else:
            label = meta['crystal_type']
            
        # B. Randomly Sample Nodes
        all_nodes = list(G.nodes())
        if len(all_nodes) == 0: continue
            
        # Sample nodes to keep dataset size manageable
        selected_nodes = random.sample(all_nodes, min(len(all_nodes), SAMPLES_PER_GRAPH))
        
        for node_id in selected_nodes:
            # Extract neighborhood (Ego Graph)
            ego = nx.ego_graph(G, node_id, radius=NEIGHBOR_RADIUS)
            
            # Prepare GraKeL data structures
            gk_labels = {}
            
            # Get features for nodes in this tiny subgraph
            node_ids = list(ego.nodes())
            
            # BUILD VECTORS FOR THE SUBGRAPH
            # We use the same helper function here
            raw_feats = [get_node_feature_vector(G.nodes[n]) for n in node_ids]
            
            # Transform to discrete codes
            discrete_codes = discretizer.transform(raw_feats).astype(int)
            
            # Create a string label for GraKeL: e.g. "5-19-0-2"
            for i, n_id in enumerate(node_ids):
                label_str = "-".join(map(str, discrete_codes[i]))
                gk_labels[n_id] = label_str
            
            # Convert edges to list of tuples
            gk_edges = list(ego.edges())
            
            # Store
            final_subgraphs.append(Graph(gk_edges, node_labels=gk_labels))
            final_labels.append(label)

    print(f"   > Done. Created {len(final_subgraphs)} subgraphs.")
    return final_subgraphs, final_labels

In [34]:
# 1. Load Data
print("Loading pickle file...")
with open(RAW_DATA_PATH, 'rb') as f:
    data = pickle.load(f)

graphs = data['graphs']
metadata = data['metadata']

# 2. Run the Processing Pipeline
X, y = process_graphs(graphs, metadata)

# 3. Quick Debug Inspection
print("\n--- DEBUG INSPECTION ---")
print(f"Total Samples: {len(X)}")
print(f"Sample Label: {y[0]}")
print(f"Sample Graph Labels (first 3 nodes):")
first_graph_labels = X[0].get_labels(purpose='dictionary')
# Print first few key-value pairs
print(dict(list(first_graph_labels.items())[:3]))

Loading pickle file...
Step 1: Collecting features from all graphs...
   > Collected 375375 feature vectors.
Step 2: Discretizing into 10 bins...
Step 3: Extracting subgraphs...
   > Done. Created 660 subgraphs.

--- DEBUG INSPECTION ---
Total Samples: 660
Sample Label: sc
Sample Graph Labels (first 3 nodes):
{2619: '9-0-3-8-6-8', 2169: '9-0-3-8-6-8', 2409: '9-0-3-8-6-8'}


In [35]:
# Cell 4: Training and Evaluation

print(f"--- STARTING TRAINING ON {len(X)} SUBGRAPHS ---")

# 1. Split Data 
# Stratify ensures we have the same % of FCC/BCC in train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED
)
print(f"Train size: {len(X_train)} | Test size: {len(X_test)}")

# 2. Initialize WL-OA Kernel
# n_iter=4 means the algorithm looks 4 hops away (neighbors of neighbors...)
# normalize=True is CRITICAL for comparison across different subgraph sizes
print("Initializing WL-OA Kernel...")
gk = WeisfeilerLehmanOptimalAssignment(n_iter=4, normalize=True, n_jobs=-1)

# 3. Compute Gram Matrices
print("Computing Training Kernel Matrix (this allows the SVM to see graph similarity)...")
# This is the most time-consuming step. It compares every training graph to every other training graph.
K_train = gk.fit_transform(X_train)

print("Computing Test Kernel Matrix...")
K_test = gk.transform(X_test)

# 4. Train SVM
print("Fitting SVM Classifier...")
# C=10 allows for a slightly more complex decision boundary (less regularization)
# class_weight='balanced' fixes issues if you have fewer 'Disordered' samples than 'FCC'
clf = SVC(kernel='precomputed', C=10.0, class_weight='balanced')
clf.fit(K_train, y_train)

# 5. Predict and Evaluate
print("Predicting on Test Set...")
y_pred = clf.predict(K_test)

print("\n--- RESULTS ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))

--- STARTING TRAINING ON 660 SUBGRAPHS ---
Train size: 528 | Test size: 132
Initializing WL-OA Kernel...
Computing Training Kernel Matrix (this allows the SVM to see graph similarity)...
Computing Test Kernel Matrix...
Fitting SVM Classifier...
Predicting on Test Set...

--- RESULTS ---
Accuracy: 0.4848

Detailed Classification Report:
              precision    recall  f1-score   support

  Disordered       0.41      1.00      0.59        48
         bcc       1.00      0.24      0.38        21
         fcc       1.00      0.14      0.25        21
         hcp       1.00      0.19      0.32        21
          sc       1.00      0.19      0.32        21

    accuracy                           0.48       132
   macro avg       0.88      0.35      0.37       132
weighted avg       0.79      0.48      0.42       132



Loading raw data from: /home/npkamath/553Project/553ProjectGraphKernels/data/raw/crystal_graphs_dataset.pkl
Processing graphs (this may take 1-2 minutes)...
Step 1: Collecting features from all graphs...
Step 2: Discretizing features into 20 bins...
Step 3: Relabeling and extracting subgraphs...
   > Extraction complete. Created 264 subgraphs.
✅ Done! You have 264 subgraphs ready for experimentation.
